In [ ]:
# 2_贝叶斯分析.ipynb
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta

# 模拟数据：两台机器生产的产品
np.random.seed(42)

# 1. 定义先验信息
# 机器M1（旧机器）和M2（新机器）的生产比例
production_ratio = {'M1': 2/3, 'M2': 1/3}  # 2:1

# 不良品率
defect_rate = {'M1': 0.02, 'M2': 0.005}

# 2. 贝叶斯公式计算
def bayesian_inference(defective_item=True):
    """计算后验概率"""
    # 全概率公式：P(不良品)
    P_defective = (production_ratio['M1'] * defect_rate['M1'] + 
                   production_ratio['M2'] * defect_rate['M2'])
    
    if defective_item:
        # 已知抽到不良品，计算来自M1的概率
        P_M1_given_defective = (production_ratio['M1'] * defect_rate['M1']) / P_defective
        return P_M1_given_defective
    else:
        # 已知抽到良品，计算来自M1的概率
        P_good = 1 - P_defective
        P_M1_given_good = (production_ratio['M1'] * (1 - defect_rate['M1'])) / P_good
        return P_M1_given_good

# 3. 计算结果
P_defective_total = production_ratio['M1'] * defect_rate['M1'] + production_ratio['M2'] * defect_rate['M2']
P_M1_defective = bayesian_inference(defective_item=True)

print("=== 贝叶斯分析结果 ===")
print(f"机器生产比例: M1={production_ratio['M1']:.1%}, M2={production_ratio['M2']:.1%}")
print(f"不良品率: M1={defect_rate['M1']:.1%}, M2={defect_rate['M2']:.1%}")
print(f"随机抽到不良品的总概率: {P_defective_total:.3%}")
print(f"抽到不良品时，它来自旧机器M1的概率: {P_M1_defective:.3%}")

# 4. 敏感性分析：不同生产比例下的后验概率
ratios = np.linspace(0.1, 0.9, 9)  # M1的生产比例从10%到90%
posterior_probs = []

for ratio in ratios:
    production_ratio_temp = {'M1': ratio, 'M2': 1-ratio}
    P_defective_temp = production_ratio_temp['M1'] * defect_rate['M1'] + production_ratio_temp['M2'] * defect_rate['M2']
    P_posterior_temp = (production_ratio_temp['M1'] * defect_rate['M1']) / P_defective_temp
    posterior_probs.append(P_posterior_temp)

# 5. 可视化
plt.figure(figsize=(10, 6))

plt.subplot(2, 2, 1)
x = np.linspace(0, 1, 1000)
plt.plot(x, beta.pdf(x, 2, 100), 'b-', label='M1先验:Beta(2,100)')
plt.plot(x, beta.pdf(x, 1, 200), 'r-', label='M2先验:Beta(1,200)')
plt.xlabel('不良品率p')
plt.ylabel('概率密度')
plt.title('机器不良品率的先验分布')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 2)
# 模拟观察到数据后的后验分布
n_samples = 100
defect_M1 = int(n_samples * defect_rate['M1'])
defect_M2 = int(n_samples * defect_rate['M2'])

plt.plot(x, beta.pdf(x, 2 + defect_M1, 100 + n_samples - defect_M1), 'b-', label='M1后验')
plt.plot(x, beta.pdf(x, 1 + defect_M2, 200 + n_samples - defect_M2), 'r-', label='M2后验')
plt.xlabel('不良品率p')
plt.ylabel('概率密度')
plt.title('观察到数据后的后验分布')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 3)
plt.plot(ratios, posterior_probs, 'o-', linewidth=2)
plt.xlabel('机器M1的生产比例')
plt.ylabel('P(M1|不良品)')
plt.title('生产比例对后验概率的影响')
plt.grid(True, alpha=0.3)

plt.subplot(2, 2, 4)
# 模拟不同不良品率下的影响
base_rates = np.linspace(0.001, 0.05, 10)
posterior_curve = []
for rate in base_rates:
    defect_rate_temp = {'M1': rate, 'M2': rate/4}
    P_defective_temp = production_ratio['M1'] * defect_rate_temp['M1'] + production_ratio['M2'] * defect_rate_temp['M2']
    P_posterior_temp = (production_ratio['M1'] * defect_rate_temp['M1']) / P_defective_temp
    posterior_curve.append(P_posterior_temp)

plt.plot(base_rates, posterior_curve, 's-', linewidth=2)
plt.xlabel('机器M1的不良品率')
plt.ylabel('P(M1|不良品)')
plt.title('不良品率对后验概率的影响')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('贝叶斯分析.png', dpi=300, bbox_inches='tight')
plt.show()